In [1]:
from libs.SatelliteEnvironment import *
from libs.DRLAgents.DQNAgent import *
from libs.Constants import *

import numpy as np
import torch 

C:\Users\minstone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
env = CircularOrbit(
    satellite_num=1000
)

env.reset()

state = env.get_state()

num_states = state.shape[0]
orbit_num  = env.orbit_num
sat_num    = env.satellite_num
num_action = orbit_num * sat_num

print("ORBIT NUM : ", orbit_num)
print("SAT NUM : ", sat_num)
print("STATE NUM : ", num_states)
print("ACTION NUM : ", num_action)

agent      = DQNAgent(num_states = num_states,
                      num_action = num_action)

agent.train()

losses = []
rewards = []

episodes = 10000
for episode in range(episodes):
    env.reset()

    ep_reward = 0
    ep_loss = 0
    step = 0
    while True:
        step += 1
        state = env.get_state()

        action = agent.get_action(state)

        reward, done, info = env.step(action)

        next_state = env.get_state() # 
            
        agent.append_sample(state=state, 
                            action=action, 
                            reward=reward, 
                            next_state=next_state,
                            done=done) # 
        
        ep_reward += reward

        if step >= MEMORY_CAPACITY:

            loss = agent.train_model()
            ep_loss += loss
            
            if(step % 100 == 0):
                agent.update_target()
            
            if done:
                agent.write_summary(ep_reward, ep_loss, step, episode)
                print("episode: {} , the episode reward is {}".format(step, ep_reward))
                break
        if done:
            break


ORBIT NUM :  2
SAT NUM :  8
STATE NUM :  36
ACTION NUM :  16


KeyboardInterrupt: 